In [2]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# time
import datetime

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

#### Cancer 생성
- baseline 1기, 1기에 진단받지 않은 사람들 대상으로 2-7기에 진단받으면 Cancer 종류명 = 1, 아니면 Cancer 종류명 = 0

#### Cancer variables list
- 폐암 : LCA, LCAAG
- 위암 : GCA, GCAAG
- 간암 : HCCCA, HCCCAAG
- 대장암 : COLCA, COLCAAG
- 췌장암 : PACA, PACAAG
- 자궁암 : UTCA, UTCAAG
- 유방암 : BRCA, BRCAAG
- 갑상선암 : THYCA, THYCAAG
- 전립선암 : PROCA, PROCAAG
- 담낭 및 기타 담도암 : GALLCA, GALLCAAG

In [3]:
final = pd.read_csv('data\\MME_final.csv', encoding = 'euc-kr', low_memory = False)
final

,기수,NIHID,SEX,VISITALL,PHYSTB,PHYSIT,PHYACTL,PHYACTM,PHYACTH,AEROBFQ,...,FMCDMAG,FMHEA,FMFHEA,FMFHEAAG,FMMHEA,FMMHEAAG,FMBHEA,FMBHEAAG,FMCHEA,FMCHEAAG
0,A01,NIH1604000095,2,6,1.0,5.0,7.0,6.0,4.0,NaN,...,NaN,1.0,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN
1,A01,NIH1604000171,1,7,0.0,3.0,8.0,8.0,0.0,NaN,...,NaN,1.0,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN
2,A01,NIH1604000338,1,7,2.0,5.0,3.0,0.0,7.0,NaN,...,NaN,1.0,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN
3,A01,NIH1604000362,2,7,0.0,5.0,4.0,2.0,2.0,NaN,...,NaN,1.0,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN
4,A01,NIH1604000424,1,4,8.0,5.0,8.0,8.0,0.0,NaN,...,NaN,1.0,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70205,A07,NIH1604999643,2,7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN
70206,A07,NIH1604999732,1,6,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN
70207,A07,NIH1604999772,1,6,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN
70208,A07,NIH1604999929,2,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN


In [4]:
# cancer df

lca = final[['기수', 'NIHID', 'LCA', 'LCAAG', 'AGE']].copy()
gca = final[['기수', 'NIHID', 'GCA', 'GCAAG', 'AGE']].copy()
hccca = final[['기수', 'NIHID', 'HCCCA', 'HCCCAAG', 'AGE']].copy()
colca = final[['기수', 'NIHID', 'COLCA', 'COLCAAG', 'AGE']].copy()
paca = final[['기수', 'NIHID', 'PACA', 'PACAAG', 'AGE']].copy()
utca = final[['기수', 'NIHID', 'UTCA', 'UTCAAG', 'AGE']].copy()
brca = final[['기수', 'NIHID', 'BRCA', 'BRCAAG', 'AGE']].copy()
thyca = final[['기수', 'NIHID', 'THYCA', 'THYCAAG', 'AGE']].copy()
proca = final[['기수', 'NIHID', 'PROCA', 'PROCAAG', 'AGE']].copy()
gallca = final[['기수', 'NIHID', 'GALLCA', 'GALLCAAG', 'AGE']].copy()

In [ ]:
# # 자동화 위해 column 명 바꾸기 (temporarily)

# lca.rename(columns = {'LCA' : 'CANCER', 'LCAAG': 'CANCER_AGE'}, inplace = True)
# gca.rename(columns = {'GCA' : 'CANCER', 'GCAAG': 'CANCER_AGE'}, inplace = True)
# hccca.rename(columns = {'HCCCA' : 'CANCER', 'HCCCAAG': 'CANCER_AGE'}, inplace = True)
# colca.rename(columns = {'COLCA' : 'CANCER', 'COLCAAG': 'CANCER_AGE'}, inplace = True)
# paca.rename(columns = {'PACA' : 'CANCER', 'PACAAG': 'CANCER_AGE'}, inplace = True)
# utca.rename(columns = {'UTCA' : 'CANCER', 'UTCAAG': 'CANCER_AGE'}, inplace = True)
# brca.rename(columns = {'BRCA' : 'CANCER', 'BRCAAG': 'CANCER_AGE'}, inplace = True)
# thyca.rename(columns = {'THYCA' : 'CANCER', 'THYCAAG': 'CANCER_AGE'}, inplace = True)
# proca.rename(columns = {'PROCA' : 'CANCER', 'PROCAAG': 'CANCER_AGE'}, inplace = True)
# gallca.rename(columns = {'GALLCA' : 'CANCER', 'GALLCAAG': 'CANCER_AGE'}, inplace = True)

In [5]:
# cancer list

cancer_list = [lca, gca, hccca, colca, paca, utca, brca, thyca, proca, gallca]

In [6]:
# 1차때 진단받은 사람 제거

new_cancer_list = []
for cancer in cancer_list: 
    cancer_1st = cancer.loc[(cancer['기수'] == 'A01') & (cancer.iloc[:, 2] == 2)]
    unique_id_1st = cancer_1st['NIHID'].unique().tolist()

    for id in unique_id_1st: 
        cancer = cancer.loc[cancer['NIHID'] != id].reset_index(drop = True)

    new_cancer_list.append(cancer)

In [7]:
new_cancer_list

[        기수          NIHID  LCA  LCAAG   AGE
 0      A01  NIH1604000095  1.0    NaN  43.0
 1      A01  NIH1604000171  1.0    NaN  42.0
 2      A01  NIH1604000338  1.0    NaN  69.0
 3      A01  NIH1604000362  1.0    NaN  48.0
 4      A01  NIH1604000424  1.0    NaN  47.0
 ...    ...            ...  ...    ...   ...
 70156  A07  NIH1604999643  1.0    NaN  55.0
 70157  A07  NIH1604999732  1.0    NaN  75.0
 70158  A07  NIH1604999772  1.0    NaN  55.0
 70159  A07  NIH1604999929  1.0    NaN  53.0
 70160  A07  NIH1604999956  NaN    NaN   NaN
 
 [70161 rows x 5 columns],
         기수          NIHID  GCA  GCAAG   AGE
 0      A01  NIH1604000095  1.0    NaN  43.0
 1      A01  NIH1604000171  1.0    NaN  42.0
 2      A01  NIH1604000338  1.0    NaN  69.0
 3      A01  NIH1604000362  1.0    NaN  48.0
 4      A01  NIH1604000424  1.0    NaN  47.0
 ...    ...            ...  ...    ...   ...
 70037  A07  NIH1604999643  1.0    NaN  55.0
 70038  A07  NIH1604999732  1.0    NaN  75.0
 70039  A07  NIH1604999772

In [8]:
new_cancer_list[0]

,기수,NIHID,LCA,LCAAG,AGE
0,A01,NIH1604000095,1.0,NaN,43.0
1,A01,NIH1604000171,1.0,NaN,42.0
2,A01,NIH1604000338,1.0,NaN,69.0
3,A01,NIH1604000362,1.0,NaN,48.0
4,A01,NIH1604000424,1.0,NaN,47.0
...,...,...,...,...,...
70156,A07,NIH1604999643,1.0,NaN,55.0
70157,A07,NIH1604999732,1.0,NaN,75.0
70158,A07,NIH1604999772,1.0,NaN,55.0
70159,A07,NIH1604999929,1.0,NaN,53.0


In [9]:
# ID별 2-7기 df 생성 후 Cancer 진단여부 열 추가

diagnosed_cancer_list = []

for cancer in new_cancer_list: 
    finalDf = pd.DataFrame()
    unique_id = cancer['NIHID'].unique().tolist()

    for id in unique_id: 
        count = 0
        df = pd.DataFrame(cancer.loc[cancer['NIHID'] == id].reset_index(drop = True))

        for i in range(1, len(df) - 1):    # 2기부터 7기까지 검사
            if df.iloc[i][2] == 2: 
                count += 1
                break; 
        if count > 0: 
            df['realCancer'] = 1
        else: 
            df['realCancer'] = 0

        finalDf = pd.concat([finalDf, df], axis = 0)
        finalDf.reset_index(drop = True, inplace = True)

    diagnosed_cancer_list.append(finalDf)

In [10]:
diagnosed_cancer_list[1]

,기수,NIHID,GCA,GCAAG,AGE,realCancer
0,A01,NIH1604000095,1.0,NaN,43.0,0
1,A02,NIH1604000095,1.0,NaN,45.0,0
2,A03,NIH1604000095,1.0,NaN,47.0,0
3,A04,NIH1604000095,1.0,NaN,NaN,0
4,A05,NIH1604000095,1.0,NaN,51.0,0
...,...,...,...,...,...,...
70037,A03,NIH1604999956,NaN,NaN,NaN,0
70038,A04,NIH1604999956,NaN,NaN,NaN,0
70039,A05,NIH1604999956,NaN,NaN,NaN,0
70040,A06,NIH1604999956,NaN,NaN,NaN,0


In [11]:
len(diagnosed_cancer_list)

10

In [133]:
# 전체 한꺼번에 안돌아감 (너무 많음)
# 하나씩 cancer에 담아서 실행할것

# cancer = diagnosed_cancer_list[0]
# cancer = diagnosed_cancer_list[1]
# cancer = diagnosed_cancer_list[2]
# cancer = diagnosed_cancer_list[3]
# cancer = diagnosed_cancer_list[4]
# cancer = diagnosed_cancer_list[5]
# cancer = diagnosed_cancer_list[6]
# cancer = diagnosed_cancer_list[7]
# cancer = diagnosed_cancer_list[8]
cancer = diagnosed_cancer_list[9]

In [134]:
cancer

,기수,NIHID,GALLCA,GALLCAAG,AGE,realCancer
0,A01,NIH1604000095,1.0,NaN,43.0,0
1,A02,NIH1604000095,1.0,NaN,45.0,0
2,A03,NIH1604000095,1.0,NaN,47.0,0
3,A04,NIH1604000095,1.0,NaN,NaN,0
4,A05,NIH1604000095,1.0,NaN,51.0,0
...,...,...,...,...,...,...
70198,A03,NIH1604999956,NaN,NaN,NaN,0
70199,A04,NIH1604999956,NaN,NaN,NaN,0
70200,A05,NIH1604999956,NaN,NaN,NaN,0
70201,A06,NIH1604999956,NaN,NaN,NaN,0


In [135]:
cancer['realCancer'].value_counts()

0    70182
1       21
Name: realCancer, dtype: int64

In [136]:
finalDf2 = pd.DataFrame()
unique_id = cancer['NIHID'].unique().tolist()

for id in unique_id: 
    df = pd.DataFrame(cancer.loc[cancer['NIHID'] == id].reset_index(drop = True))
    first_age = cancer['AGE'][cancer['기수'] == 'A01']
    last_exam_age = 0

    for i in range(len(df) - 1, -1, -1): 
        if (pd.isnull(cancer['AGE'][i]) == False): 
            last_exam_age = cancer['AGE'][i]
            break; 

    for i in range(1, len(df)): 
        if (pd.isnull(df.iloc[:, 3][i]) == False): 
            diagnosed_age = df.iloc[:, 3][i]
            break; 

    if df['realCancer'][0] == 0:    # 진단 받지 않은 사람이라면
        time_l = last_exam_age - first_age  # TIME은 마지막 검진일 - 1차 검진일
    if df['realCancer'][0] == 1:    # 진단 받은 사람이라면
        time_l = diagnosed_age - first_age

    df['TIME'] = time_l
    finalDf2 = pd.concat([finalDf2, df], axis = 0)
    finalDf2.reset_index(drop = True, inplace = True)

In [137]:
finalDf2

,기수,NIHID,GALLCA,GALLCAAG,AGE,realCancer,TIME
0,A01,NIH1604000095,1.0,NaN,43.0,0,12.0
1,A02,NIH1604000095,1.0,NaN,45.0,0,NaN
2,A03,NIH1604000095,1.0,NaN,47.0,0,NaN
3,A04,NIH1604000095,1.0,NaN,NaN,0,NaN
4,A05,NIH1604000095,1.0,NaN,51.0,0,NaN
...,...,...,...,...,...,...,...
70198,A03,NIH1604999956,NaN,NaN,NaN,0,NaN
70199,A04,NIH1604999956,NaN,NaN,NaN,0,NaN
70200,A05,NIH1604999956,NaN,NaN,NaN,0,NaN
70201,A06,NIH1604999956,NaN,NaN,NaN,0,NaN


In [138]:
k = finalDf2[['NIHID', 'realCancer', 'TIME']]
k

,NIHID,realCancer,TIME
0,NIH1604000095,0,12.0
1,NIH1604000095,0,NaN
2,NIH1604000095,0,NaN
3,NIH1604000095,0,NaN
4,NIH1604000095,0,NaN
...,...,...,...
70198,NIH1604999956,0,NaN
70199,NIH1604999956,0,NaN
70200,NIH1604999956,0,NaN
70201,NIH1604999956,0,NaN


In [139]:
k.drop_duplicates(inplace = True, subset = ['NIHID'])

<ipython-input-139-4cc8615d917b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  k.drop_duplicates(inplace = True, subset = ['NIHID'])


In [140]:
k.reset_index(inplace = True, drop = True)
k

,NIHID,realCancer,TIME
0,NIH1604000095,0,12.0
1,NIH1604000171,0,12.0
2,NIH1604000338,0,12.0
3,NIH1604000362,0,12.0
4,NIH1604000424,0,12.0
...,...,...,...
10024,NIH1604999643,0,12.0
10025,NIH1604999732,0,12.0
10026,NIH1604999772,0,12.0
10027,NIH1604999929,0,12.0


In [141]:
k['realCancer'].value_counts()

0    10026
1        3
Name: realCancer, dtype: int64

In [142]:
k.rename(columns = {'realCancer' : 'GALLCA'}, inplace = True)

C:\Users\user\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [143]:
k

,NIHID,GALLCA,TIME
0,NIH1604000095,0,12.0
1,NIH1604000171,0,12.0
2,NIH1604000338,0,12.0
3,NIH1604000362,0,12.0
4,NIH1604000424,0,12.0
...,...,...,...
10024,NIH1604999643,0,12.0
10025,NIH1604999732,0,12.0
10026,NIH1604999772,0,12.0
10027,NIH1604999929,0,12.0


In [144]:
k.to_csv('data\\Cancer\\GALLCA.csv', encoding = 'euc-kr', index = False)